## <center> Progetto Deep Learning - CycleGAN - Alessandro Soccol 60/79/00057</center>
### <center>Abstract</center>
Una cycleGAN non richiede un dataset di immagini accoppiate. Se vogliamo tradurre da arancie a mele, non abbiamo bisogno di un dataset di training formato da arancie che sono state convertite in mele. Questo permette di tradurre, per esempio, dipinti in fotografie. Tuttavia è un'architettura con molti limiti e utile in specifici casi molto poco complessi.

> **Informazione:** Questo notebook è frutto del tutorial in questo link https://machinelearningmastery.com/cyclegan-tutorial-with-keras/ e di articoli scientifici cui è fatta una panoramica a fine notebook e che ho usato per cercare di dare risposta a domande quali: perchè non è adatto al nostro task una CycleGAN? Quali alternative ci sono? Cosa avrei potuto fare per raggiungere gli obiettivi che mi ero prefissato?

### <center> Un minimo di teoria </center>
L'architettura del modello è formata da due generatori, un generatore (A) genera immagini del primo dominio, il secondo generatore (B) genera immagini del secondo dominio. I generatori si occupano di fare una traduzione dell'immagine, ciò significa che la generazione dell'immagine è condizionata all'immagine di input che nel nostro caso è quella dell'altro dominio. Il generatore A, per esempio, prende un'immagine del dominio B in input e il generatore B prende un'immagine del dominio A in input.\
Ogni generatore ha un corrispondente modello discriminatore. Il discriminatore (A) prende le immagini reali del dominio (A) e immagini generate dal generatore (A) e predice se sono immagini reali o fake. Il secondo discriminatore (B) prende immagini reali del dominio B e immagini generate dal generatore (B) e predice se sono immagini reali o fake.

    Domain-A -> Discriminator-A -> [Real/Fake]
    Domain-B -> Generator-A -> Discriminator-A -> [Real/Fake]
    Domain-B -> Discriminator-B -> [Real/Fake]
    Domain-A -> Generator-B -> Discriminator-B -> [Real/Fake]

Come una GAN, il generatore cerca di "imbrogliare" il discriminatore generando immagini che sono sempre più realistiche, mentre il discriminatore impara a rilevare meglio le immagini false.\
I generatori traducono versioni più ricostruite delle immagini di input dal dominio di origine e questo è fatto confrontando l'immagine di output del generatore con l'immagine originale. Il passaggio di un'immagine da entrambi i generatori è definito come ciclo. Ogni coppia di generatori è addestrata a riprodurre l'immagine di partenza, questo è definito come cycle-consistency.

    Domain-B -> Generator-A -> Domain-A -> Generator-B -> Domain-B
    Domain-A -> Generator-B -> Domain-B -> Generator-A -> Domain-A

Una CycleGAN, per definizione, possiamo riassumerla con tre concetti fondamentali:
- Unpaired image-to-image translation (Come scritto sopra, non abbiamo bisogno di un dataset di immagini accoppiate)
- Adversarial Loss (Come nelle DCGAN tradizionali già viste a lezione)
- Cycle-Consistency Loss

![.](https://i.ibb.co/kQHJgRq/1.png)\
Il modello contiene due funzioni di mapping $G:X\rightarrow Y$ e $F:Y\rightarrow X$ con i rispettivi discriminatori $D_Y$ e $D_X$. $D_Y$ incoraggia $G$ a tradurre gli elementi del dominio $X$ in dati indistinguibili dal dominio $Y$, viceversa per $D_X$ e $F$. Per regolarizzare ulteriormente il modello, oltre all'adversarial loss, vengono introdotte due *cycle consistency loss* il cui obiettivo è fare in modo che se traduco da un dominio ad un altro, poi torno al dominio di partenza, devo arrivare al dato da cui sono partito. La cycle consistency loss si divide quindi in *forward cycle-consistency loss* in cui $x\rightarrow G(x)\rightarrow F(G(x))\approx x$ (figura sopra (b)) e *backward cycle-consistency loss* in cui $y\rightarrow F(y)\rightarrow G(F(y))\approx y$ (figura sopra (c)).\
**<center>Formalizzando</center>**
L'obiettivo è imparare delle funzioni di mapping tra due domini $X$ ed $Y$, dati i loro esempi appartenenti al dominio, $\{x_i\}_{i=1}^N$ per cui $x_i\in X$ e $\{y_j\}_{j=1}^M$ per cui $y_j\in Y$. Il modello include due funzioni di mapping $G:X\rightarrow Y$ e $F:Y\rightarrow X$. In più vengono aggiunti due discriminatori $D_X$ e $D_Y$, in cui $D_X$ ha l'obiettivo di distinguere tra $y$ e le immagini tradotte $\{F(y)\}$, mentre $D_Y$ ha l'obiettivo di distinguere tra $\{y\}$ e $\{G(x)\}$.\
La funzione obiettivo contiene un'adversarial loss per fare il matching della distribuzione delle immagini generate con la distribuzione dei dati nel dominio target (Come una GAN) e una cycle consistency loss per fare in modo che i mapping imparati $G$ e $F$ non si contraddicano a vicenda.
**<center>Adversarial Loss</center>**
L'adversarial loss è applicata ad entrambe le funzioni di mapping, per esempio nel caso della funzione di mapping $G$ e il suo discriminatore $D_Y$, abbiamo $$\mathcal{L}_{GAN}(G,D_Y,X,Y)=\mathbb{E}_{y\sim p_{data}(y)}[\log D_Y(y)]+\mathbb{E}_{x\sim p_{data}(x)}[\log (1-D_Y (G(x)))]$$
In cui $G$ cerca di generare immagini $G(x)$ che sembrano simili alle immagini del dominio $Y$, mentre $D_Y$ ha l'obiettivo di distinguere tra le immagini tradotte $G(x)$ e gli esempi originali del dominio $y$. $G$ ha l'obiettivo di minimizzare la funzione mentre $D$ cerca di massimizzarla. Da quest'ultima frase si può intuire che l'obiettivo è cercare un equilibrio, non abbiamo una convergenza della funzione. quindi riassumento 
$$\min_G \max_{D_Y} \mathcal{L}_{GAN}(G,D_Y,Y,X)$$
Facciamo la stessa cosa per la funzione di mapping $F$ e il discriminatore $D_X$, quindi
$$\min_F \max_{D_X} \mathcal{L}_{GAN}(F,D_X,Y,X)$$
**<center>Cycle Consistency Loss</center>**
<center>Perchè usiamo un'altra loss? non potevamo usare l'adversarial loss e basta?</center>

Quando una rete ha una grande capacità, può mappare lo stesso insieme di immagini di input ad una permutazione casuale di immagini nel dominio target in cui qualsiasi dei mapping imparati nel dominio target può indurre una distribuzione di output che corrisponde alla distribuzione target. Quindi l'adversarial loss da sola non può garantire che la funzione imparata possa mappare un input $x_i$ all'output desiderato $y_j$. Quindi per ridurre lo spazio delle possibili funzioni di mapping si vuole che le funzioni di mapping siano cycle-consistent, per ogni immagine $x$ del dominio $X$ il ciclo di traduzione dell'immagine dovrebbe essere in grado di riportare $x$ all'immagine originale, ossia $x\rightarrow G(x)\rightarrow F(G(x))\approx x$, questa è la forward cycle consistency. Allo stesso modo, per ogni immagine $y$ del dominio $Y$, $G$ ed $F$ devono soddisfare la backward cycle consistency tale per cui $y\rightarrow F(y)\rightarrow G(F(y))\approx y$.\
Questo viene fatto usando una cycle consistency loss 
$$\mathcal{L}_{cyc}(G,F)=\mathbb{E}_{x\sim p_{data}(x)}[||F(G(x))-x||_1]+\mathbb{E}_{y\sim p_{data}(y)}[||G(F(y))-y||_1]$$

Infine possiamo scrivere la funzione di loss finale come 
$$\mathcal{L}(G,F,D_X,D_Y)=\mathcal{L}_{GAN}(G,D_Y,X,Y)+\mathcal{L}_{GAN}(F,D_X,Y,X)+\lambda \mathcal{L}_{cyc}(G,F)$$
**<center>Limitazioni</center>**
Una CycleGAN può fare cambiamenti minimi all'input, tant'è che la CycleGAN fallisce quando trova per esempio un cavallo con sopra una persona, oppure quando si vuole per esempio trasformare da cani a gatti. Un ricercatore giapponese è riuscito ad usare una cycleGAN ed avere risultati accettabili nel trasformare da gatti a cani, dice di aver usato una cycle loss più piccola (suppongo che intenda il valore di $\lambda$ nella funzione di loss) ed un discriminatore locale+globale (suppongo il focus fosse su l'uso di un discriminatore locale che riesce a catturare informazioni locali come tratti della faccia di cani e gatti), tuttavia non ha rilasciato un'implementazione ne molte informazioni a riguardo. <a href="https://qiita.com/itok_msi/items/b6b615bc28b1a720afd7">articolo ricercatore giapponese</a>\
![Fallimento CycleGAN](https://i.ibb.co/9ThDZVK/2.png)
> **Informazione**: Le informazioni qua sopra, di teoria, sono prese dall'articolo originale. Per tenere il notebook il più snello possibile ho limitato la quantità di informazioni. 
> <a href="https://arxiv.org/abs/1703.10593">Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks</a>

**<center>Struttura della directory</center>**

catvsdogs_cgan_small  
-- *A -> cat  
-- *B -> dog  

La struttura del dataset deve essere cosi perchè ricordiamoci che passiamo da un Dominio A ad un dominio B e viceversa. é necessario definire bene i domini

In [1]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True) # force_remount mi serve perchè colab non si accorge che i file cambiano nel drive, in questo modo fa il remounting

Mounted at /content/drive/


### <center>Informazioni Preliminari</center>
Il colab si connetterà a google drive, è importante avere 2 directory nel proprio drive
- Dataset
- Checkpoint

In checkpoint ci devono essere le cartelle dei pesi dei vari modelli. Devono essere 6.\
In dataset ci deve essere il file `catsvsdogs_64.npz`
> **Importante**: All'interno di ogni cartella riguardante i pesi di un modello, ci sono file "nascosti" che iniziano con il punto. Assicurarsi che quei file siano presenti nelle cartelle dei pesi di ogni modello, altrimenti darà errore. 
> **Copiando ed incollando la cartella nel drive potrebbe non copiare questi file!**


In [2]:
# Può creare le cartelle cosi. è necessario inserire manualmente i file richiesti all'interno accedendo a google drive nel link nel file README.txt
!mkdir /content/drive/MyDrive/Dataset
!mkdir /content/drive/MyDrive/Checkpoint

Una volta eseguita la cella sopra per la prima volta, è necessario caricare i file che trova in allegato sui pesi dei modelli e sul dataset compresso!

In [3]:
training_phase=True # Vuole addestrare? allora a True.
create_compressed_dataset=False # Vuoi creare il dataset in forma compressa (.npz)? [Richiede un pò di tempo!]
load_trained_models=False # Vuoi caricare un modello addestrato precedentemente?
show_some_data=False # Mostrami alcuni dati del dataset a caso. Non voglio perchè mi rallenta e basta l'esecuzione totale del codice
iterazione="050600" # iterazione del modello che stai caricando, da impostare (e utile) solo se load_trained_models è true, andrà a cercare il file dell'iterazione che hai scelto su gdrive
loaded=0 # quando carichi il modello, non riprende ad iterare da 0 ma da dove si era fermato, viene modificato in seguito, non toccare!
directory="/content/drive/MyDrive/" # gdrive directory
directoryDataset="/content/drive/MyDrive/Dataset/" # gdrive directory dataset
directoryCheckpoint="/content/drive/MyDrive/Checkpoint/" # gdrive checkpoint

In [4]:
from os import listdir
import numpy as np
from numpy import asarray
from numpy import vstack
from keras.utils import img_to_array
from keras.utils import load_img
from numpy import savez_compressed

# carico tutte le immagini in una directory, in memoria
def load_images(path, size=(64,64)):
    data_list = list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # store
        data_list.append(pixels)
    return np.asarray(data_list)

if create_compressed_dataset:
    # dataset path
    path = directoryDataset+"catsvsdogs_cgan_small_CycleGAN/"
    # load dataset A
    dataA1 = load_images(path + 'trainA/')
    dataAB = load_images(path + 'testA/')
    dataA = vstack((dataA1, dataAB)) # Notare come usiamo tutti i dati di training che abbiamo! non ci interessa se sono di test. In quanto usiamo il modello successivamente per aumentare i dati e passare da un dominio ad un altro
    # vstack è una concatenazione dei dati sul primo asse.
    print('Loaded dataA: ', dataA.shape)
    # load dataset B
    dataB1 = load_images(path + 'trainB/')
    dataB2 = load_images(path + 'testB/')
    dataB = vstack((dataB1, dataB2)) # Notare come usiamo tutti i dati di training che abbiamo! non ci interessa se sono di test
    # vstack è una concatenazione dei dati sul primo asse.
    print('Loaded dataB: ', dataB.shape)
    # save as compressed numpy array
    filename = f'{directoryDataset}catsvsdogs_64.npz' # comprimiamo il dataset per usarlo meglio con Numpy, infatti successivamente lo caricheremo con load di numpy
    savez_compressed(filename, dataA, dataB) # creiamo il dataset in maniera compressa .npz e lo salviamo 
    print('Saved dataset: ', filename)

Mostriamo alcuni dati, solo se `show_some_data` definita prima è True.

In [5]:
from numpy import load
from matplotlib import pyplot
if show_some_data:
    # load and plot the prepared dataset
    # load the dataset
    data = load(f'{directoryDataset}catsvsdogs_64.npz')
    dataA, dataB = data['arr_0'], data['arr_1']
    print('Loaded: ', dataA.shape, dataB.shape)
    # plot source images
    n_samples = 3
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(dataA[i].astype('uint8'))
    # plot target image
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(dataB[i].astype('uint8'))
    pyplot.show()

## <center> Costruzione della CycleGAN </center>
`tensorflow_addons` è necessario per usare un layer chiamato InstanceNormalization non incluso in tensorflow classico.

In [6]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


La instanceNormalization è uguale alla BatchNormalization vista a lezione, solamente anzichè considerare un batch consideriamo le singole istanze.

In [7]:
from tensorflow_addons.layers import InstanceNormalization
# define layer
layer = InstanceNormalization(axis=-1) # axis = -1 per fare in modo che le caratteristiche siano normalizzate per ogni feature map.

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


### <center> Discriminatore </center>
Notare i blocchi formati da layer convolutivi, InstanceNormalization e LeakyReLU.\
La notazione C64, C128, C256 ecc. è definita come segue:
- `C` indica un blocco formato da Convoluzione, BatchNormalizatione e Leaky ReLU
- il numero subito dopo indica il numero di filtri
- I layer convolutivi, quando scriviamo quella notazione, hanno una kernel size 4x4 e uno stride 2x2
Più info sulla notazione: <a href="https://arxiv.org/pdf/1611.07004.pdf">Image-to-Image Translation with Conditional Adversarial Networks ; Appendix 6, (6.1) </a> è una notazione molto usata quando si usano modelli con blocchi di questo tipo; l'ho ritrovata in diversi articoli. Può variare ed essere per esempio definita come `c7s1-k` in cui abbiamo un blocco formato da Conv-InstanceNorm-ReLU 7x7 con `k` filtri e stride 1, `dk` indica un blocco come scritto prima ma 3x3 con `k`filtri e stride 2. e altro che si può trovare nell'articolo <a href="https://arxiv.org/pdf/2002.10102.pdf">GANHopper, sezione 3.1</a>

In [8]:
import tensorflow as tf
# define the discriminator model
def define_discriminator(image_shape):
    # weight initialization
    init = tf.keras.initializers.RandomNormal(stddev=0.02,seed=1337)
    # source image input
    in_image = tf.keras.Input(shape=image_shape)
    # C64
    d = tf.keras.layers.Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    # C128
    d = tf.keras.layers.Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    # C256
    d = tf.keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    # C512
    d = tf.keras.layers.Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = tf.keras.layers.Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization(axis=-1)(d)
    d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
    # patch output
    patch_out = tf.keras.layers.Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    # define model
    model = tf.keras.Model(in_image, patch_out)
    # compile model


    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5])
    return model

### <center> Generatore </center>
Il generatore ha un'architettura encoder-decoder. Il modello prende un'immagine reale come per esempio un cane e genera un'immagine di un gatto. Fa questo facendo prima di tutto il downsampling o l'encoding dell'immagine facendola passare per un collo di bottiglia, dopodiché interpreta la codifica con un numero di connessioni residue (ResNet) seguita da una serie di layer che fanno l'upsampling o la decodifica della rappresentazione, alla dimensione dell'immagine che ci serve in output.

Definiamo un blocco residual network, visto a lezione. Questi blocchi formati da due layer CNN 3x3 il cui input del blocco è concatenato all'output del blocco.\
Crea due blocchi Convoluzione-InstanceNormalization con filtri 3x3 e stride 1x1 senza un'attivazione ReLU dopo il secondo blocco.

Il generatore è addestrato tramite il relativo modello discriminatore. I generatori sono addestrati per fare in modo di minimizzare la loss predetta dal discriminatore per le immagini generate, identificate come reali. Questo è fatto con l'adversarial loss, in questo modo il generatore è spinto a generare immagini sempre migliori.\
Il generatore è inoltre aggiornato sulla base di quanto è efficace nella generazione dell'immagine di input quando viene usato con gli altri generatori, da qua viene il cycle-loss.\
Infine un generatore restituisce un'immagine in output senza traduzione quando gli viene data un'immagine del dominio di destinazione, questo avviene grazie all'identity loss.\
Ogni generatore è ottimizzato attraverso la combinazione di quattro output con 4 funzioni di loss

    - Adversarial loss (L2 or mean squared error).
    - Identity loss (L1 or mean absolute error).
    - Forward cycle loss (L1 or mean absolute error).
    - Backward cycle loss (L1 or mean absolute error).




In [9]:
# generator a Residual Network block
def resnet_block(n_filters, input_layer):
    # weight initialization
    init = tf.keras.initializers.RandomNormal(stddev=0.02,seed=1337)
    # first layer convolutional layer
    g = tf.keras.layers.Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # second convolutional layer
    g = tf.keras.layers.Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    # concatenate merge channel-wise with input layer
    g = tf.keras.layers.Concatenate()([g, input_layer])
    return g

In [10]:
# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
    # weight initialization
    init = tf.keras.initializers.RandomNormal(stddev=0.02)
    # image input
    in_image = tf.keras.Input(shape=image_shape)
    # c7s1-64
    g = tf.keras.layers.Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # d128
    g = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # d256
    g = tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # R256
    # Residual Network
    for _ in range(n_resnet):
        g = resnet_block(256, g)
    # u128
    g = tf.keras.layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # u64
    g = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = tf.keras.layers.Activation('relu')(g)
    # c7s1-3
    g = tf.keras.layers.Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    out_image = tf.keras.layers.Activation('tanh')(g)
    # define model
    model = tf.keras.Model(in_image, out_image)
    return model

### <center> Composite model </center>
Per addestrare i generatori e discriminatori definiamo un modello che al suo interno ha i generatori e discriminatori. Questo è fatto per addestrare ogni generatore il quale è responsabile di aggiornare i suoi pesi anche se è necessario condividere i pesi con il suo discriminatore e l'altro generatore.

Il discriminatore è connesso all'output del generatore per fare in modo che classifichi le immagini generate come reali o fake. Un secondo input del composite model è definito come un'immagine dal dominio target, anzichè il dominio sorgente, che il generatore si aspetta di dare in output senza la traduzione per l'identity mapping. Successivamente la forward cycle loss comporta il collegamento dell'uscita del generatore all'altro generatore che ricostruirà l'immagine di partenza. Infine, la backward cycle loss coinvolge l'immagine del dominio di destinazione usata per l'identity mapping che viene fatta passare anche attraverso l'altro generatore, la cui uscita è collegata al nostro generatore principale come ingresso e produce una versione ricostruita di quell'immagine dal dominio di partenza.\
In sintesi, il composite model ha due input per le immagini reali del dominio A e B, quattro uscite per l'output del discriminatore ossia l'immagine generata dall'identity, l'immagine generata dal forward cycle e l'immagine generata dal backward cycle.\
Per il composite model vengono aggiornati solo i pesi del primo modello o del generatore principale e ciò avviene tramite la somma ponderata di tutte le funzioni di perdita. Alla perdita del ciclo viene attribuito un peso maggiore (10) rispetto alla perdita avversaria, come descritto nel paper originale, e la perdita dell'identità viene sempre utilizzata con una ponderazione pari alla metà di quella della perdita del ciclo (5). Come definito nell'implementazione del paper.

**<center>Cosa ci facciamo con il composite model?</center>**\
Dobbiamo creare un composite model per ogni generatore, per esempio per cani verso gatti e per gatti verso cani.\
\
*Generatore A - Composite Model da Cani (B) a Gatti (A)*

    Adversarial Loss: Domain-B -> Generator-A -> Domain-A -> Discriminator-A -> [real/fake]
    Identity Loss: Domain-A -> Generator-A -> Domain-A
    Forward Cycle Loss: Domain-B -> Generator-A -> Domain-A -> Generator-B -> Domain-B
    Backward Cycle Loss: Domain-A -> Generator-B -> Domain-B -> Generator-A -> Domain-A

*Generatore B - Composite model da Gatti (A) a Cani (B)*

    Adversarial Loss: Domain-A -> Generator-B -> Domain-B -> Discriminator-B -> [real/fake]
    Identity Loss: Domain-B -> Generator-B -> Domain-B
    Forward Cycle Loss: Domain-A -> Generator-B -> Domain-B -> Generator-A -> Domain-A
    Backward Cycle Loss: Domain-B -> Generator-A -> Domain-A -> Generator-B -> Domain-B


> Ricordiamoci che nelle prime celle il dominio A corrisponde ai gatti e il dominio B ai cani!


In [11]:
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    # ensure the model we're updating is trainable
    g_model_1.trainable = True
    # mark discriminator as not trainable
    d_model.trainable = False
    # mark other generator model as not trainable
    g_model_2.trainable = False
    # discriminator element
    input_gen = tf.keras.Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)
    # identity element
    input_id = tf.keras.Input(shape=image_shape)
    output_id = g_model_1(input_id)
    # forward cycle
    output_f = g_model_2(gen1_out)
    # backward cycle
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)
    # define model graph
    model = tf.keras.Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    # define optimization algorithm configuration
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)
    # compile model with weighting of least squares loss and L1 loss
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
    return model

Funzione per caricare i dati di training, vengono scalati tra -1 e 1; scalare i dati è una best practice che si trova in tutti i paper riguardanti le GAN.

In [12]:
# load and prepare training images
def load_real_samples(filename):
    # load the dataset
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

`generate_real_samples` prende un numpy array per un dominio input e restituisce il numero di immagini in maniera random che si vuole.

In [13]:
import numpy as np
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

`generate_fake_samples` genera un campione, dato un generatore e il campione di immagini reali dal dominio sorgente

In [14]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
    # generate fake instance
    X = g_model.predict(dataset)
    # create 'fake' class labels (0)
    y = np.zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

`save_models` salva i pesi dei modelli

In [15]:
# salvo tutti i pesi del modello nel file, lo faccio per caricare i dati e continuare l'addestramento da dove viene interrotto per google colab
def save_models(step, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA,d_model_AtoB, d_model_BtoA):
    # save the first generator model
    filename1 = f'{directory}Checkpoint/g_model_AtoB_%06d_tf/' % (step+1)
    g_model_AtoB.save_weights(filename1)
    # save the second generator model
    filename2 = f'{directory}Checkpoint/g_model_BtoA_%06d_tf/' % (step+1)
    g_model_BtoA.save_weights(filename2)
    # save the third generator model
    filename3 = f'{directory}Checkpoint/c_model_AtoB_%06d_tf/' % (step+1)
    c_model_AtoB.save_weights(filename3)
    # save the fourth generator model
    filename4 = f'{directory}Checkpoint/c_model_BtoA_%06d_tf/' % (step+1)
    c_model_BtoA.save_weights(filename4)
    # save the fifth generator model
    filename5 = f'{directory}Checkpoint/d_model_A_%06d_tf/' % (step+1)
    d_model_AtoB.save_weights(filename5)
    # save the sixth generator model
    filename6 = f'{directory}Checkpoint/d_model_B_%06d_tf/' % (step+1)
    d_model_BtoA.save_weights(filename6)
    print('>Saved: %s %s %s %s %s %s' % (filename1, filename2,filename3,filename4, filename5,filename6))

`summarize_performance` mostra come vengono traslate le immagini da un dominio ad un altro.

In [16]:
from matplotlib import pyplot
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
    # select a sample of input images
    X_in, _ = generate_real_samples(trainX, n_samples, 0)
    # generate translated images
    X_out, _ = generate_fake_samples(g_model, X_in, 0)
    # scale all pixels from [-1,1] to [0,1]
    X_in = (X_in + 1) / 2.0
    X_out = (X_out + 1) / 2.0
    # plot real images
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_in[i])
    # plot translated image
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_out[i])
    # save plot to file
    filename1 = f'{directory}Checkpoint/%s_generated_plot_%06d.png' % (name, (step+1))
    pyplot.savefig(filename1)
    pyplot.close()
    print("IMMAGINE SALVATA")

`update_image_pool` serve per creare un pool di 50 immagini che vengono cambiate di volta in volta

In [17]:
from random import random
# update image pool for fake images
def update_image_pool(pool, images, max_size=5):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = np.random.randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

Funzione di Training. La `batch_size` è impostata ad un'immagine come nel paper originale.\
Viene preso un batch di immagini reali da ogni dominio, viene generato un batch di immagini da ogni dominio. Le immagini fake sono usate per aggiornare il pool di immagini fake di ogni discriminatore.



In [18]:
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, loaded):
	# define properties of the training run
	n_epochs, n_batch, = 100, 1 # 100 epoche, batch di una immagine ciascuno
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	print("N_PATH - TRAIN FUCN",d_model_A.output_shape)
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		g_loss2*=0.1
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		g_loss1*=0.1
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('[ iterazione -> %d | epoch -> %d ] dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+loaded+1,(i+loaded+1)/len(trainA), dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i+loaded, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i+loaded, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 1) == 0:
			print("\n\nSto Salvando i pesi \n\n")
			if load_trained_models:
				# Salviamo i modelli per poterli ricaricare.
				save_models(i+loaded, g_model_AtoB, g_model_BtoA, d_model_A, d_model_B,c_model_AtoB,c_model_BtoA)
			else:
				save_models(i, g_model_AtoB, g_model_BtoA, d_model_A, d_model_B,c_model_AtoB,c_model_BtoA)

Definizione generatori, discriminatori e caricamento dei dati. Nel caso in cui ci fossero dei pesi da caricare, vengono definiti i modelli e gli vengono caricati i pesi. Ho scelto di salvare e caricare i pesi perchè mi sembrava più intuitivo da capire, piuttosto che salvarmi i modelli nel formato .h5 che da problemi (dice che il modello non è compilato).

In [19]:
from tensorflow import keras
from numpy import load
dataset = load_real_samples(f'{directoryDataset}catsvsdogs_64.npz') # da modificare, bisogna ricreare il dataset
# Carico immagini
print('Loaded', dataset[0].shape, dataset[1].shape)
# definisco la shape dell'input sulla base del dataset caricato
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
if load_trained_models: # carico modello precedente e riparto da dove mi sono fermato, usato perchè cosi se cadono le sessioni di colab free riesco a ricaricarle e ripartire
    loaded=int(iterazione)
    g_model_AtoB.load_weights(f'{directoryCheckpoint}g_model_AtoB_{iterazione}_tf/')
    g_model_BtoA.load_weights(f'{directoryCheckpoint}g_model_BtoA_{iterazione}_tf/')
    d_model_A.load_weights(f'{directoryCheckpoint}d_model_A_{iterazione}_tf/')
    d_model_B.load_weights(f'{directoryCheckpoint}d_model_B_{iterazione}_tf/')
    c_model_AtoB.load_weights(f'{directoryCheckpoint}c_model_AtoB_{iterazione}_tf/')
    c_model_BtoA.load_weights(f'{directoryCheckpoint}c_model_BtoA_{iterazione}_tf/')

# Addestramento del modello, solo se training_phase è a true!
if training_phase: # magari non voglio fare il training!
  train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, loaded)

Loaded (2200, 64, 64, 3) (2200, 64, 64, 3)


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


N_PATH - TRAIN FUCN (None, 4, 4, 1)
1/1 [==============================] - 2s 2s/step
[ iterazione -> 1 | epoch -> 0 ] dA[0.812,0.479] dB[1.068,0.548] g[1.897,1.866]
1/1 [==============================] - 0s 21ms/step
[ iterazione -> 2 | epoch -> 0 ] dA[2.984,0.536] dB[0.582,0.492] g[1.724,1.912]
1/1 [==============================] - 0s 22ms/step
[ iterazione -> 3 | epoch -> 0 ] dA[0.438,0.859] dB[0.692,1.221] g[2.094,1.909]
1/1 [==============================] - 0s 24ms/step
[ iterazione -> 4 | epoch -> 0 ] dA[0.429,0.796] dB[2.304,0.913] g[1.657,1.675]
1/1 [==============================] - 0s 21ms/step
[ iterazione -> 5 | epoch -> 0 ] dA[1.454,0.438] dB[1.256,0.600] g[1.699,1.703]
1/1 [==============================] - 0s 28ms/step
[ iterazione -> 6 | epoch -> 0 ] dA[0.686,12.632] dB[1.191,1.055] g[1.681,1.662]
1/1 [==============================] - 0s 21ms/step
[ iterazione -> 7 | epoch -> 0 ] dA[1.695,24.127] dB[1.069,18.075] g[1.868,1.775]
1/1 [==============================] - 

KeyboardInterrupt: ignored

## <center>Traduciamo le immagini da un dominio ad un altro</center>
Solamente dopo aver fatto l'addestramento o dopo aver caricato i pesi del modello!


### <center>Carichiamo il dataset</center>

In [ ]:
A_data, B_data = load_real_samples(f'{directoryDataset}catsvsdogs_64.npz') # prendo dei dati dal dataset
print('Loaded', A_data.shape, B_data.shape)

### <center>Definiamo e carichiamo i pesi dei modelli</center>


In [ ]:
# generator: A -> B
model_AtoB = define_generator(image_shape)
# generator: B -> A
model_BtoA = define_generator(image_shape)

In [ ]:
model_AtoB.load_weights(f'{directoryCheckpoint}g_model_AtoB_{iterazione}_tf/')
model_BtoA.load_weights(f'{directoryCheckpoint}g_model_BtoA_{iterazione}_tf/')

### <center>Testiamo la cycleGAN da un dominio all'altro e viceversa</center>

Creiamo una funzione che prende un'immagine qualsiasi nel dataset

In [ ]:
def select_sample(dataset, n_samples):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	return X

Scegliamo un'immagine random dal dominio A (Gatti), per tradurla nel dominio B (Cani) usando il generatore A -> B

In [ ]:
# plot A->B->A
A_real = select_sample(A_data, 1)
B_generated  = model_AtoB.predict(A_real)
A_reconstructed = model_BtoA.predict(B_generated)

Facciamo il plot delle tre immagini: immagine reale, immagine tradotta nell'altro dominio, immagine ricostruita nel dominio originale.

In [ ]:
# plot the image, the translation, and the reconstruction
def show_plot(imagesX, imagesY1, imagesY2):
	images = vstack((imagesX, imagesY1, imagesY2))
	titles = ['Real', 'Generated', 'Reconstructed']
	# scale from [-1,1] to [0,1]
	images = (images + 1) / 2.0
	# plot images row by row
	for i in range(len(images)):
		# define subplot
		pyplot.subplot(1, len(images), 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(images[i])
		# title
		pyplot.title(titles[i])
	pyplot.show()

In [ ]:
show_plot(A_real, B_generated, A_reconstructed)

Possiamo fare la stessa cosa da Cani a Gatti

In [ ]:
B_real = select_sample(B_data, 1)
A_generated  = model_BtoA.predict(B_real)
B_reconstructed = model_AtoB.predict(A_generated)

In [ ]:
show_plot(B_real, A_generated, B_reconstructed)

### <center>Approfondimenti sul perchè non è stata usata, limitazioni, possibili soluzioni</center>

La cycleGAN non è stata utilizzata perchè, come viene citato in diversi articoli che spiegherò tra poco, una cycleGAN non performa bene quando ci sono in gioco variazioni di forma o geometriche.

Ci sono numerose altre alternative all'uso della CycleGAN per l'obiettivo che mi ero prefissato, una di queste è <a href="https://arxiv.org/abs/2002.10102" >GANHopper: Multi-Hop GAN for Unsupervised Image-to-Image Translation</a> che potrebbe essere riassunto come "Quale cane apparirebbe simile dato un gatto?".\
![GANHopper](https://i.ibb.co/6P98Hmj/1.png)\
Una GANHopper quello che fa è, anzichè fare traduzioni DIRETTE da un dominio ad un altro (per esempio quello che fa la cycleGAN) che non permette l'acquisizione delle variazioni geometriche necessarie, trasformare da un dominio ad un altro facendo degli "hop" ossia passando a traduzioni intermedie dell'immagine. Anche in questo caso non abbiamo coppie di immagini accoppiate da un dominio all'altro (unpaired image-to-image translation).
Tutti gli Hop sono prodotti usando un singolo generatore lungo ogni direzione, oltre alle loss tipiche della GAN e CycleGAN, abbiamo un nuovo discriminatore chiamato "Hybrid discriminator" che è addestrato per classificare le immagini generate nei vari hop. Una GANHopper eccelle nella traduzione di immagini in cui ci sono variazioni geometriche come nel passaggio da cani a gatti e viceversa, sarebbe potuta essere forse il modello adatto al nostro obiettivo. Nell'articolo è citato esplicitamente come la CycleGAN non riesca a trasformare le caratteristiche geometriche ma solamente alterazioni della texture e colore a livello di pixel. Una GANHopper è costruita sopra una CycleGAN.\
Per esempio:\
Una rete four-hop per tradurre da cani a gatti produce tre immagini intermedie
- 25% gatto, 75% cane
- 50% gatto, 50% cane
- 75% gatto, 25% cane
- 0% gatto, 100% cane <- Final Hop!

La capacità della rete non eccede quella di una cycleGAN e l'hybrid discriminator è addestrato solamente su immagini reali per valutare le immagini generate nei vari hop. Vengono introdotte due nuove loss oltre quelle tipiche della cycleGAN, la hybrid loss e la smoothness loss.

- Hybrid Loss : per valutare il grado di appartenenza di un'immagine a uno dei domini di ingresso
- Smoothness Loss: che regola ulteriormente le transizioni dell'immagine per garantire che un'immagine generata nella sequenza di hop non si discosti molto dall'immagine precedente.

![GanHopper 2 - Hop](https://i.ibb.co/XWnz0sk/2.png)

Il framework multi-hop è formato da due generatori uguali alla CycleGAN e tre discriminatori, due dei quali anch'essi uguali a quelli della CycleGAN. Il terzo discriminatore è quello citato sopra, l'hybrid discriminator.\
Una comparativa con altri modelli tra cui la cycleGAN

![Ganhopper, comparativa con altri modelli](https://i.ibb.co/SBZGHJy/3.png)
\
\
Un altro articolo <a href="https://link.springer.com/chapter/10.1007/978-3-030-70665-4_92">A Survey on Data Augmentation Methods Based on GAN in Computer Vision</a> spiega come la DA tradizionale ha un impatto molto limitato nel avere una diversità maggiore nel dataset, le cycleGAN riescono a incrementare la diversità del dataset trasferendo lo stile dell'immagine tra immagini. Viene scritto inoltre che i metodi basati su GAN per la DA si dividono in tre tipi, i primi generano esempi da uno spazio latente (per esempio CGAN, ACGAN, TripleGAN, infoGAN ecc.), i secondi fanno una traduzione dell'immagine e i terzi producono immagini usando una strategia avversaria. Viene spiegato anche il modello Pix2Pix su cui si basa la cycleGAN, che altro non è che una CGAN in cui anzichè condizionare sulle label, condizioniamo sull'immagine in input per restituire immagini target; tant'è che la cycleGAN non ha bisogno di dati accoppiati tra i due domini di traduzione, mentre Pix2Pix si. Una cycleGAN viene in aiuto per la risoluzione dei limiti del modello CGAN Pix2Pix. Per risolvere i limiti di Pix2Pix e cycleGAN, riguardo al fatto che traducono tra DUE domini, è stato creato il modello StarGAN che permette di tradurre immagini da più domini. Infine nella conclusione è specificato come i metodi di DA basati su GAN sono efficaci in molti campi, tuttavia è necessario avere molti dati (cosa che noi non avevamo).
\
\
Un altro articolo <a href="https://arxiv.org/pdf/2003.00273.pdf">Reusing Discriminators for Encoding:
Towards Unsupervised Image-to-Image Translation</a> mostra come diversi sforzi sono stati fatti per trasformare immagini da un dominio ad un altro per diversi task come coloramento di un'immagine, editing immagini, migliorare risoluzione ecc senza la necessità di coppie di immagini tra un dominio ed un altro, la cycleGAN viene citata come un esempio di un modello che viene usato per fare questo. In questo articolo, si chiedono se si può ripensare il ruolo di ogni componente in una GAN (prevalenetemente, anche se non indicato, solamente il ruolo di generatore e discriminatore). In questo articolo viene proposto di riusare il discriminatore per fare l'encoding, si usano i primi layer nel discriminatore come encoder del dominio target, in questa maniera riusciamo ad avere un'architettura più compatta dato che l'encoder diventa parte del discriminatore e non abbiamo bisogno di un componente indipendente per l'encoding. In più, nel momento in cui si deve fare inferenza, la parte usata per l'encoding del discriminatore è tenuta per fare inferenza. In questo modo l'encoder è addestrato in maniera più efficace, infatti in genere l'addestramento dell'encoder è fatto facendo la backpropagation dei gradienti dal generatore, collegando l'encoder al discriminatore, l'encoder è addestrato direttamente attraverso la loss del discriminatore. 
Dato che l'encoder e il discriminatore si sovrappongono, questo comporta un'instabilità se applichiamo un training setting tradizionale (adversarial), l'encoder come parte della traduzione viene addestrato per minimizzare e allo stesso tempo appartiene al discriminatore e viene quindi anche addestrato per massimizzare. Per risolvere quest'ultimo problema, l'addestramento è disaccoppiato. L'addestramento dell'encoder è associato solo al discriminatore indipendentemente dal generatore; gli esperimenti hanno mostrato che questo disaccoppiamento favorisce notevolmente l'addestramento. Viene mostrato come nella traduzione da cane a gatto, si hanno performance più del doppio migliori rispetto ad una CycleGAN.
![.](https://i.ibb.co/23tW4rd/3.png)

Un fatto che secondo me ha senso da far notare è come le cycleGAN possono essere molto utili in campo medico. Infatti, come citato nel blog creato dai creatori della cycleGAN <a href="https://junyanz.github.io/CycleGAN/">cycleGAN Blog</a>, una CycleGAN può trasformare un'immagine in un altra in cui magari è presente un tumore per mostrare come sarebbe, o viceversa nel caso in cui si volesse mostrare la non presenza di un tumore. Oppure può avere altri usi come il colorare le immagini, come citato sempre nel blog ufficiale. Ci sono anche altri usi per cui un modello come le CycleGAN risulta essere molto utile, nonostante le sue limitazioni.

Ci sono altri articoli, ho citato i principali che penso abbiano maggior importanza.

Il codice in questione è stato scelto da diversi tutorial online per leggibilità sulla base di quello visto a lezione di teoria. La maggior parte erano implementazioni in PyTorch o tensorflow ma non facilmente spiegabili come questo.